<a href="https://colab.research.google.com/github/jeongukjae/Artificial-Intelligence-Assignments/blob/master/ch3/1_wine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch

from torch import optim
from torch.nn import Module, Linear, CrossEntropyLoss, functional
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

In [0]:
# load dataset
wine = load_wine()

wine_data = wine.data[0:130]
wine_target = wine.target[0:130]

In [12]:
train_x, test_x, train_y, test_y = train_test_split(wine_data, wine_target, test_size=0.2)

print(f"length of train_x : {len(train_x)}")
print(f"length of train_y : {len(train_y)}")
print(f"length of test_x : {len(test_x)}")
print(f"length of test_y : {len(test_y)}")

length of train_x : 104
length of train_y : 104
length of test_x : 26
length of test_y : 26


In [13]:
# create tensor data from trainig data
train_x_tensor = torch.from_numpy(train_x).float()
train_y_tensor = torch.from_numpy(train_y).long()

# create tensor data from test data
test_x_tensor = torch.from_numpy(test_x).float()
test_y_tensor = torch.from_numpy(test_y).long()

# print shape of trainig data
print(f"shape of train_x_tensor: {train_x_tensor.shape}")
print(f"shape of train_y_tensor: {train_y_tensor.shape}")
print(f"shape of test_x_tensor: {test_x_tensor.shape}")
print(f"shape of test_y_tensor: {test_y_tensor.shape}")

shape of train_x_tensor: torch.Size([104, 13])
shape of train_y_tensor: torch.Size([104])
shape of test_x_tensor: torch.Size([26, 13])
shape of test_y_tensor: torch.Size([26])


In [14]:
# create tensor dataset for train
tensor_dataset_for_train = TensorDataset(train_x_tensor, train_y_tensor)
print(f"first data of tensor_dataset_for_train: \n{tensor_dataset_for_train[0]}")

# create minibatch of tensor_dataset_for_train
train_loader = DataLoader(tensor_dataset_for_train, batch_size=16, shuffle=True)

first data of tensor_dataset_for_train: 
(tensor([1.3340e+01, 9.4000e-01, 2.3600e+00, 1.7000e+01, 1.1000e+02, 2.5300e+00,
        1.3000e+00, 5.5000e-01, 4.2000e-01, 3.1700e+00, 1.0200e+00, 1.9300e+00,
        7.5000e+02]), tensor(1))


In [0]:
# create neural net
class NeuralNet(Module):
  def __init__(self):
    super(NeuralNet, self).__init__()
    self.fully_connected1 = Linear(13, 96)
    self.fully_connected2 = Linear(96, 96)
    self.fully_connected3 = Linear(96, 96)
    self.fully_connected4 = Linear(96, 96)
    self.fully_connected5 = Linear(96, 96)
    self.fully_connected6 = Linear(96, 2)

  def forward(self, x):
    x = functional.relu(self.fully_connected1(x))
    x = functional.relu(self.fully_connected2(x))
    x = functional.relu(self.fully_connected3(x))
    x = functional.relu(self.fully_connected4(x))
    x = functional.relu(self.fully_connected5(x))
    x = self.fully_connected6(x)
    return functional.log_softmax(x, dim=1)

  
model = NeuralNet()

In [0]:
# error function
criterion = CrossEntropyLoss()

# use SGD optimizer and 0.01 learning rate
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [17]:
# start training
for epoch in range(300):
  total_loss = 0
  
  # take training data
  for data_x, data_y in train_loader:
    train_x_variable, train_y_variable = Variable(data_x), Variable(data_y)
    optimizer.zero_grad()
    
    output = model(train_x_variable)
    loss = criterion(output, train_y_variable)
    
    loss.backward()
    optimizer.step()
    total_loss += loss.data.item()

  if (epoch + 1) % 50 == 0:
    print(f"epoch {epoch + 1} , total loss is {total_loss}")

epoch 50 , total loss is 2.8746326714754105
epoch 100 , total loss is 2.425212226808071
epoch 150 , total loss is 1.8903180286288261
epoch 200 , total loss is 1.7058668732643127
epoch 250 , total loss is 1.3640936389565468
epoch 300 , total loss is 2.592431478202343


In [18]:
test_x_variable, test_y_variable = Variable(test_x_tensor), Variable(test_y_tensor)

result = torch.max(model(test_x_variable).data, 1)[1]

accuracy = sum(test_y_variable.data.numpy() == result.numpy()) / len(test_y_variable.data.numpy())

print(f"accuracy : {accuracy}")

accuracy : 0.8846153846153846
